In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(df.head(20))